# 제주도 명소 - 인스타그램 그롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
chromedriver = '/Users/baekj/Downloads/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

## 1. Instagram에 접속하여 로그인

In [3]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [4]:
with open('password.txt') as f:
    password = f.read(10)

In [5]:
len(password)

10

In [6]:
email = 'bbakganzi_90'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [7]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [8]:
# 로그인 정보 저장 - 나중에 하기
driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
time.sleep(1)

In [9]:
# 알림 설정 - 나중에 하기
try:        # 있으면 클릭 없으면 pass
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

## 2. 제주도맛집으로 검색

In [10]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [11]:
driver.get(url)
time.sleep(5)       # 충분한 time.sleep시간을 줘야함

## 3. 첫번째 게시글 열기

In [12]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

## 4. 게시글 정보 가져오기

In [13]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [16]:
# 1) 본문 내용 가져오기
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐히 들어간 순두부 찌개다. #신해바라기분식 #제주도맛집뜨끈한 국물과 두부 한 숟갈을 떠먹으면 화끈한 매운맛과 뜨거운 열기가 식욕을 자극한다. (신라면보다 조금 매운 정도?)맵기 조절이 가능하기에 취향에 따라 먹을 수 있다. 매운 걸 먹지 못하는 일행이 있다면, 백순두부도 좋다. 더 담백하고 맑은 국물로 두부맛에 집중할 수 있다. 보글보글 끓어나오는 콩비지 느낌의 순두부 찌개와 오징어 젓갈이 기억에 남는 제주도 로컬 맛집이다. 📍 제주 일도1동 1342-4◾순두부찌개 ₩7,000🏷 피웨 부캐 노포 간판 수집가 @nopo__joa🏷 미식가들의 맛집 모음집 @e.a.t.club#순두부찌개 #국밥맛집 #제주맛집 #제주시맛집 #피그웨이브_제주'

In [31]:
# 2) 본문 내용에서 해시태그 가져오기   '#[^\s#,\\]+'
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags


['#신해바라기분식', '#제주도맛집뜨끈한', '#순두부찌개', '#국밥맛집', '#제주맛집', '#제주시맛집', '#피그웨이브_제주']

In [21]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-18'

In [28]:
# 4) 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like=0
like

0

In [27]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30xS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [30]:
row = [content, tags, date, like, place]
row

['42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐히 들어간 순두부 찌개다. #신해바라기분식 #제주도맛집뜨끈한 국물과 두부 한 숟갈을 떠먹으면 화끈한 매운맛과 뜨거운 열기가 식욕을 자극한다. (신라면보다 조금 매운 정도?)맵기 조절이 가능하기에 취향에 따라 먹을 수 있다. 매운 걸 먹지 못하는 일행이 있다면, 백순두부도 좋다. 더 담백하고 맑은 국물로 두부맛에 집중할 수 있다. 보글보글 끓어나오는 콩비지 느낌의 순두부 찌개와 오징어 젓갈이 기억에 남는 제주도 로컬 맛집이다. 📍 제주 일도1동 1342-4◾순두부찌개 ₩7,000🏷 피웨 부캐 노포 간판 수집가 @nopo__joa🏷 미식가들의 맛집 모음집 @e.a.t.club#순두부찌개 #국밥맛집 #제주맛집 #제주시맛집 #피그웨이브_제주',
 ['#신해바라기분식', '#제주도맛집뜨끈한', '#순두부찌개', '#국밥맛집', '#제주맛집', '#제주시맛집', '#피그웨이브_제주'],
 '2021-08-18',
 0,
 '']

## 5. 다음 게시글로 이동

In [33]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [71]:
# 6을 하기 위해 게시글 닫기
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

## 6. 여러개의 게시글 가져오기

In [45]:
from tqdm.notebook import tqdm

In [59]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like=0

    try:
        place = soup.select('div.M30xS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return[content, tags, date, like, place]

### - 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [62]:
keyword = '제주여행'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [63]:
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [50]:
#제주도맛집
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

,content,date,like,place,tags
0,".💙 구좌 ""제주모랑"" 💙사장님들이 손수 만든 수제소품들이 가득~좋은향이 가득한 구...",2021-08-18,"1,027",,"[#제주모랑, #제주소품샵, #세화소품샵, #제주기념품, #제주기념품샵, #구좌읍소..."
1,.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰...,2021-08-18,"1,008",,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
2,그 유명한 #연돈 에 다녀왔습니다. 그런데ㅎ;;;⠀⠀⠀⠀⠀⠀⠀어마무시한 웨이팅으로 ...,2021-08-18,"e_foodie님 외 4,534명이 좋아합니",,"[#연돈, #제주도연돈, #테이블링, #등심까스, #수제볼카츠, #아토짱_제주, #..."
3,.한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋...,2021-08-17,"1,303",,"[#문도지오름방목된, #제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, ..."
4,<광고>#제주분식 https://jejubunsig.modoo.at/ 📌제주시 귀아...,2021-08-16,0,,"[#제주분식, #제주분식⠀간편하게, #모당모당, #돔베라면돔베김밥, #제주맛집, #..."


In [51]:
#제주도맛집
df.tail()

,content,date,like,place,tags
195,제주 역대급 돈까스맛집🍴뭐든 푸짐푸짐한거ㅇㅈ?쩝쩝박사님들ㄲㄲ⠀@@내최애는 고구마치즈...,2021-08-19,0,,"[#애월맛집, #애월맛집추천, #애월밥집, #애월돈까스, #애월데이트코스, #애월핫..."
196,흑돼지는 정직한돈‼매일먹어도 먹고싶은내최애맛집 제발머겅❤ 🔥⠀@@오늘갈랭?🤭⠀⠀#애...,2021-08-19,0,,"[#애월맛집, #협재맛집, #한림맛집, #애월흑돼지, #협재흑돼지, #한림흑돼지, ..."
197,신혼여행 1일차😍생각해보니 결혼식 관련해서 아무것도 안올렸넹..누가보면 결혼식 못한...,2021-08-19,23,,"[#제주도, #신혼여행, #3개월지났지만, #올린당, #지난신혼여행, #낭만없음, ..."
198,육즙대잔치라는 정직한돈👏🏻셀럽들이 인정하는곳이래안가면 손해👏🏻👏🏻👏🏻⠀@@메모해🥰...,2021-08-19,0,,"[#애월맛집, #협재맛집, #한림맛집, #애월흑돼지, #협재흑돼지, #한림흑돼지, ..."
199,흑돼지는 정직한돈‼매일먹어도 먹고싶은내최애맛집 제발머겅❤ 🔥⠀@@오늘갈랭?🤭⠀⠀#애...,2021-08-19,0,,"[#애월맛집, #협재맛집, #한림맛집, #애월흑돼지, #협재흑돼지, #한림흑돼지, ..."


In [67]:
df.to_csv('data/제주맛집.csv', index=False)

In [72]:
for keyword in ['제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(10)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(2)

    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

    df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
    df.to_csv(f'data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(5)

  0%|          | 0/200 [00:00<?, ?it/s]

In [73]:
driver.close()

### 4가지 CSV파일의 중복을 제거한 후 통합 저장

In [74]:
jeju_df = pd.DataFrame([])
for keyword in ['제주도관광', '제주맛집', '제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')
    

(196, 5)
(200, 5)
(198, 5)
